In [1]:
import nltk
import random, math, copy, os, string
from scipy.stats import pearsonr
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn  
from nltk.corpus import wordnet_ic
from nltk.wsd import lesk
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('conll2000')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
!pip install svgling


from google.colab import drive
drive.mount('/content/drive')

nltk.download('wordnet')
nltk.download('universal_tagset') 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('wordnet_ic')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
     |████████████████████████████████| 66 kB 4.5 MB/s 
Mounted at /content/drive
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[

True

In [2]:
import pandas as pd
dt = pd.read_csv('/content/drive/My Drive/content/drive/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('/content/drive/My Drive/content/drive/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)
dt.head()
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
#Document structure
sw = nltk.corpus.stopwords.words('english')
Jaccard_Dist=[] #Here we will store the Jaccard Distances between the pairs
tok=[] #Here we will keep a tokenized version of the pairs of phrases
for i in range(len(dt)):
  sent=[set(nltk.word_tokenize(dt[0][i])),set(nltk.word_tokenize(dt[1][i]))] 
  for n in [0,1]:
    sent[n]=set([w.lower() for w in sent[n] if w.lower() not in sw and w not in string.punctuation and w.isalpha()]) 
  tok.append(sent)
  Jaccard_Dist.append(jaccard_distance(sent[0],sent[1]))
  
dt['Jaccard']=Jaccard_Dist 
dt.head()

,0,1,gs,Jaccard
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50,0.692308
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00,0.285714
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25,0.727273
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50,0.250000
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00,0.000000


In [ ]:
tok=[] #Here we will keep a tokenized version of the pairs of phrases
for i in range(len(dt)):
  sent=[set(nltk.word_tokenize(dt[0][i])),set(nltk.word_tokenize(dt[1][i]))] 
  tok.append(sent)

PoS=[] #Here we will store the PoS tags

for i in range(len(dt)):
   PoS.append([set(nltk.pos_tag(tok[i][0])),set(nltk.pos_tag(tok[i][1]))])

lem=[]
lem.append(nltk.ne_chunk_sents(PoS))

print(lem)  
"""  
Lem_Dist=[]
for i in range(len(dt)):
  Lem_Dist.append(jaccard_distance(Lemmas[i][0],Lemmas[i][1]))
dt['Lemma']=Lem_Dist
dt.head()
"""

[<generator object ParserI.parse_sents.<locals>.<genexpr> at 0x7fbe5a3884d0>]


"  \nLem_Dist=[]\nfor i in range(len(dt)):\n  Lem_Dist.append(jaccard_distance(Lemmas[i][0],Lemmas[i][1]))\ndt['Lemma']=Lem_Dist\ndt.head()\n"

In [ ]:
sentence = "Mark Pedersen and John Smith are working at Google " + \
           "since 1994 for $1000 per week."

x = nltk.pos_tag(nltk.word_tokenize(sentence))

res = nltk.ne_chunk(x)
print(res)

(S
  (PERSON Mark/NNP)
  (ORGANIZATION Pedersen/NNP)
  and/CC
  (PERSON John/NNP Smith/NNP)
  are/VBP
  working/VBG
  at/IN
  (ORGANIZATION Google/NNP)
  since/IN
  1994/CD
  for/IN
  $/$
  1000/CD
  per/IN
  week/NN
  ./.)


In [ ]:

# Create the list to store Part of Speech Tags by nltk standard that wordnet deals with
Admissible_PoS=['NOUN', 'VERB', 'ADJ', 'ADV']

#Now we are going to change the Part of Speech tag to the format that Wordnet uses
pos_in_wn = {'DET':None, 'NOUN':'n', 'VERB':'v', 'ADJ':'a', 'ADV': 'r' , 'ADP':None, 'CONJ':None, 'NUM':None, 'PRT':None, 'PRON':None, '.':None, 'X':None }

#Tokenize the sentences
tokenized_sents = []
for i in range(len(dt)):
  sent = [nltk.word_tokenize(dt[0][i]),nltk.word_tokenize(dt[1][i])] 
  tokenized_sents.append(sent)
tokenized_sents

#First we apply the lesk algorithm, and keep the synsets
lesk = copy.deepcopy(tok)
for i, sent in enumerate(lesk): 
 for n in range(2):
   sent[n]=nltk.pos_tag(sent[n],tagset='universal')   
   sent[n] = set([nltk.ne_chunk(tokenized_sents[i][n], w[0],pos_in_wn[w[1]]) for w in sent[n] if pos_in_wn[w[1]] is not None])


#Calculating the Jaccard distance for the resulting synsets 
Jaccard_Lesk = []
for sent in lesk:
  Jaccard_Lesk.append(jaccard_distance(sent[0],sent[1]))
dt['Lesk']=Jaccard_Lesk 

dt.head()


TypeError: ignored

In [ ]:
print([pearsonr(dt['gs'], dt['Jaccard'])[0],pearsonr(dt['gs'], dt['Lemma'])[0], pearsonr(dt['gs'], dt['Lesk'])[0]])

[-0.45279693414784167, -0.47130316108106723, -0.3540038658213318]


In [ ]:
print([pearsonr(dt['Lesk'], dt['Jaccard'])[0]])

[0.8349367273950777]
